In [1]:
import pandas as pd

usecols = ["ID", "Start", "End", "Quantity", "Areas"]
deaths = pd.read_csv("Playdata/Playdata(Deaths).csv", usecols=usecols, sep=";")
injuries = pd.read_csv("Playdata/Playdata(Injuries).csv", usecols=usecols, sep=";")
residence = pd.read_csv("Playdata/Playdata(Residence).csv", usecols=usecols, sep=";")

pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', 0)

In [2]:
import dateparser
import shortuuid
import itertools


class Utils:
    # TODO: add validation for REAL GIDs
    def __init__(self):
        self.gadm = pd.read_csv("Database/data/gadm_world.csv")

    @staticmethod
    def get_level(GID: str) -> int:
        """Returns the level of a GID."""
        GID = f"{GID}_1" if (len(GID) == 3 and "_1" not in GID) else GID
        level = GID.split(".")
        level[-1] = level[-1][:-2]
        return len(level) - 1

    @staticmethod
    def random_short_uuid(length: int = 7) -> str:
        """Generates a short alpha-numerical UID"""
        return shortuuid.ShortUUID().random(length=length)

    @staticmethod
    def flatten(xss: list[list[str]]) -> list[str]:
        """Returns a flattened and sorted list from a list of lists"""
        return sorted([x for xs in xss for x in xs])

    def elementary_gids(self, GID: str) -> tuple[str]:
        """
        Returns a tuple of the GIDs that make up the input GID for the deepest possible level
        """
        l = GID.split(".")
        l[-1] = l[-1][:-2]

        for lvl in range(5, 0, -1):
            res = self.gadm[self.gadm[f"GID_{len(l)-1}"] == GID][f"GID_{lvl}"]
            if res.dropna().unique().tolist():
                return tuple(sorted(res.dropna().unique().tolist()))

        else:
            return tuple([GID])

    @staticmethod
    def exact_match(query: tuple[str], space: tuple[str]) -> bool:
        """Returns True if an exact match of a set of GIDs is found in a search space"""
        intersection = tuple(sorted(list(set(query) & set(space))))
        if intersection == query:
            return True
        return False

    @staticmethod
    def intersect(query: tuple[str], space: tuple[str]) -> bool:
        """Returns True if some elements of a set of GIDs is found in a search space"""
        intersection = tuple(sorted(list(set(query) & set(space))))
        if intersection:
            return True
        return False

    @staticmethod
    def get_intersection(tpl1: tuple[str], tpl2: tuple[str]) -> tuple[str]:
        """Returns the intersection between two tuples"""
        return tuple(sorted(list(set(tpl1) & set(tpl2))))

In [3]:
# set up tables
# ONLY RUN ONCE!

utils = Utils()
for i in [deaths, injuries, residence]:

    # normalize areas
    i["Areas"] = i["Areas"].apply(lambda x: x.split(",") if isinstance(x, str) else x)
    i["Areas"] = i["Areas"].apply(lambda x: tuple(sorted([y.strip() for y in x])))
    i["Level"] = i["Areas"].apply(lambda x: tuple([utils.get_level(g) for g in x]))

    # get a list of the smallest elements in each GID in Areas
    # (used later to calculate intersections)
    i["Breakdown"] = i["Areas"].apply(
        lambda x: tuple(Utils.flatten([utils.elementary_gids(g) for g in x]))
    )

    # infer min an max
    i["Min"] = i["Quantity"].apply(
        lambda x: int(x) if "-" not in str(x) else int(str(x).split("-")[0])
    )
    i["Max"] = i["Quantity"].apply(
        lambda x: int(x) if "-" not in str(x) else int(str(x).split("-")[1])
    )

    # set the inferred flag. True means a row has been added artifically. False means the information comes directly from the wikipedia article
    i["Inferred"] = False  # all numbers come directly from the article

    # normalize start and end dates
    i["Start"] = i["Start"].apply(lambda x: dateparser.parse(x).date())
    i["End"] = i["End"].apply(lambda x: dateparser.parse(x).date())

    # give each row a random UUID
    i["UUID"] = i["ID"].apply(lambda _: utils.random_short_uuid(length=4))

/var/folders/kq/p_10t47n1g3djxzly2sjfr240000gn/T/ipykernel_2757/4158414212.py:9: DtypeWarning: Columns (0,4,7,8,11,12,15,16,17,18,19,20,21,22,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  self.gadm = pd.read_csv("Database/data/gadm_world.csv")


In [4]:
deaths.sample(3)

,ID,Start,End,Quantity,Areas,Level,Breakdown,Min,Max,Inferred,UUID
22,333,2020-07-30,2020-08-05,1,"(USA.7.5_1,)","(2,)","(USA.7.5_1,)",1,1,False,ZxPL
4,111,2021-07-05,2021-07-09,9,"(CUB, USA.10_1)","(0, 1)","(CUB.1.10_1, CUB.1.11_1, CUB.1.12_1, CUB.1.13_1, CUB.1.1_1, CUB.1.2_1, CUB.1.3_1, CUB.1.4_1, CUB.1.5_1, CUB.1.6_1, CUB.1.7_1, CUB.1.8_1, CUB.1.9_1, CUB.10.1_1, CUB.10.2_1, CUB.10.3_1, CUB.10.4_1, CUB.10.5_1, CUB.10.6_1, CUB.10.7_1, CUB.10.8_1, CUB.11.10_1, CUB.11.11_1, CUB.11.12_1, CUB.11.13_1, CUB.11.1_1, CUB.11.2_1, CUB.11.3_1, CUB.11.4_1, CUB.11.5_1, CUB.11.6_1, CUB.11.7_1, CUB.11.8_1, CUB.11.9_1, CUB.12.10_1, CUB.12.11_1, CUB.12.1_1, CUB.12.2_1, CUB.12.3_1, CUB.12.4_1, CUB.12.5_1, CUB.12.6_1, CUB.12.7_1, CUB.12.8_1, CUB.12.9_1, CUB.13.10_1, CUB.13.11_1, CUB.13.12_1, CUB.13.13_1, CUB.13.14_1, CUB.13.1_1, CUB.13.2_1, CUB.13.3_1, CUB.13.4_1, CUB.13.5_1, CUB.13.6_1, CUB.13.7_1, CUB.13.8_1, CUB.13.9_1, CUB.14.1_1, CUB.14.2_1, CUB.14.3_1, CUB.14.4_1, CUB.14.5_1, CUB.14.6_1, CUB.14.7_1, CUB.14.8_1, CUB.15.1_1, CUB.15.2_1, CUB.15.3_1, CUB.15.4_1, CUB.15.5_1, CUB.15.6_1, CUB.15.7_1, CUB.15.8_1, CUB.15.9_1, CUB.16.10_1, CUB.16.11_1, CUB.16.12_1, CUB.16.13_1, CUB.16.1_1, CUB.16.2_1, CUB.16.3_1, CUB.16.4_1, CUB.16.5_1, CUB.16.6_1, CUB.16.7_1, CUB.16.8_1, CUB.16.9_1, CUB.2.10_1, CUB.2.1_1, CUB.2.2_1, CUB.2.3_1, CUB.2.4_1, CUB.2.5_1, CUB.2.6_1, CUB.2.7_1, CUB.2.8_1, CUB.2.9_1, CUB.3.1_1, ...)",9,9,False,sqZP
21,333,2020-07-30,2020-08-05,1,"(USA.30.2_1,)","(2,)","(USA.30.2_1,)",1,1,False,xZHC


In [5]:
injuries.sample(3)

,ID,Start,End,Quantity,Areas,Level,Breakdown,Min,Max,Inferred,UUID
5,333,2020-07-30,2020-08-05,1,"(USA.33.32_1,)","(2,)","(USA.33.32_1,)",1,1,False,3kPf
0,111,2021-06-30,2021-07-10,1,"(MTQ,)","(0,)","(MTQ.1.1_1, MTQ.1.2_1, MTQ.1.3_1, MTQ.1.4_1, MTQ.2.10_1, MTQ.2.11_1, MTQ.2.1_1, MTQ.2.2_1, MTQ.2.3_1, MTQ.2.4_1, MTQ.2.5_1, MTQ.2.6_1, MTQ.2.7_1, MTQ.2.8_1, MTQ.2.9_1, MTQ.3.1_1, MTQ.3.2_1, MTQ.3.3_1, MTQ.3.4_1, MTQ.3.5_1, MTQ.3.6_1, MTQ.3.7_1, MTQ.3.8_1, MTQ.3.9_1, MTQ.4.1_1, MTQ.4.2_1, MTQ.4.3_1, MTQ.4.4_1, MTQ.4.5_1, MTQ.4.6_1, MTQ.4.7_1, MTQ.4.8_1)",1,1,False,AS75
2,333,2020-07-30,2020-08-05,14,"(USA.34.8_1,)","(2,)","(USA.34.8_1,)",14,14,False,d5w5


In [6]:
def post_process(ID: int, tbl: pd.DataFrame) -> list[dict]:
    new_rows = []
    df = tbl[tbl.ID == ID].sort_values("Level", ascending=False)
    for _, row_i in df.iterrows():
        if row_i["Breakdown"] == row_i["Areas"]:
            r = row_i.to_dict()
            r["Subsets"] = float("nan")
            new_rows.append(r)
            del r
        else:
            res = tbl[(tbl.ID == ID) & (tbl.Breakdown.apply(lambda x: utils.intersect(row_i["Breakdown"], x))) & (tbl.Start >= row_i["Start"]) & (tbl.End <= row_i["End"]) & (tbl.UUID != row_i["UUID"]) & (tbl.Breakdown.apply(lambda x: utils.exact_match(x, row_i["Breakdown"]))) & (tbl.Breakdown != row_i["Breakdown"])]
            tmp_df = pd.DataFrame(new_rows)
            overlapping_rows = []

            # aggregare values, but do not count UUIDs that are a subset of a row
            if not tmp_df.empty:
                overlaps = tmp_df[(tmp_df.UUID.isin(res.UUID.to_list())) & ~(tmp_df.Subsets.isnull())]
                if not overlaps.empty:
                    overlapping_rows = utils.flatten(overlaps.Subsets.to_list())
            del tmp_df
            res = res[~res.UUID.isin(overlapping_rows)]
            if res.empty:
                r = row_i.to_dict()
                r["Subsets"] = float("nan")
                new_rows.append(r)
                del r
            elif not res.empty:
                impact_values = res[["Min", "Max"]].sum().to_dict()
                if impact_values["Min"] <= row_i["Min"] and impact_values["Max"] <= row_i["Max"]:
                    r = row_i.to_dict()
                    r["Subsets"] = tuple(sorted(res["UUID"].to_list()))
                    new_rows.append(r)
                    del r
                elif impact_values["Min"] >= row_i["Min"] or impact_values["Max"] >= row_i["Max"]:
                    # print("unacceptable")
                    # print(row_i["UUID"], "min/max:", row_i["Min"], "-", row_i["Max"], "Areas:", row_i["Areas"])
                    # print("RES")
                    # print(res)
                   
                    r = row_i.to_dict()
                    fresh_uuid = utils.random_short_uuid(length=4)
                    subset_list = res["UUID"].to_list()
                    #print("original subsets", subset_list)
                    subset_list.append(row_i["UUID"])
                    #print("added subset:", row_i["UUID"])
                    #print("final subset list", subset_list)
                    r["Subsets"] = tuple(sorted(subset_list))
                    r["Inferred"] = True
                    r["Min"] = impact_values["Min"]
                    r["Max"] = impact_values["Max"]
                    r["UUID"] = fresh_uuid
                    r["Quantity"] = f'{impact_values["Min"]}-{impact_values["Max"]}' if impact_values["Min"] != impact_values["Max"] else impact_values["Max"] 
                    new_rows.append(r)
                    #print("first r", r["ID"], r["UUID"], r["Min"], r["Max"], r["Areas"], r["Subsets"])
                    del r

                    r = row_i.to_dict()
                    new_rows.append(r)
                    r["Subsets"] = float("nan")
                    #print("second r", r["ID"], r["UUID"], r["Min"], r["Max"], r["Areas"], r["Subsets"])
                    del r
                    #print("-----")
    return new_rows

In [7]:
cols = [
    "ID",
    "Start",
    "End",
    "Quantity",
    "Areas",
    "Level",
    "Breakdown",
    "Min",
    "Max",
    "Inferred",
    "UUID",
    "Subsets",
]
pp_deaths, pp_injuries, pp_residence = (
    pd.DataFrame(columns=cols),
    pd.DataFrame(columns=cols),
    pd.DataFrame(columns=cols),
)

for idd in [111, 222, 333]:
    pp_deaths_df = pd.DataFrame(post_process(idd, deaths[deaths.ID == idd]))
    pp_deaths = pd.concat([pp_deaths, pp_deaths_df], axis=0)

    pp_injuries_df = pd.DataFrame(post_process(idd, injuries[injuries.ID == idd]))
    pp_injuries = pd.concat([pp_injuries, pp_injuries_df], axis=0)

    pp_residence_df = pd.DataFrame(post_process(idd, residence[residence.ID == idd]))
    pp_residence = pd.concat([pp_residence, pp_residence_df], axis=0)


In [8]:
pp_residence[pp_residence.ID ==  222][[n for n in pp_residence.columns if n != "Breakdown"]]

,ID,Start,End,Quantity,Areas,Level,Min,Max,Inferred,UUID,Subsets
0,222,2023-08-22,2023-08-27,2-9,"(MEX.2_1,)","(1,)",2,9,False,5e2m,NaN
1,222,2023-08-22,2023-08-27,1198,"(MEX,)","(0,)",1198,1198,False,CyjW,"(5e2m,)"
2,222,2023-08-22,2023-08-27,2-9,"(MEX,)","(0,)",2,9,True,3nyL,"(5e2m, YGeB)"
3,222,2023-08-22,2023-08-27,1,"(MEX,)","(0,)",1,1,False,YGeB,NaN


In [10]:
resid = pp_residence[pp_residence.Areas.astype(str).apply(lambda x: "USA.21" in x)]

In [11]:
overlapping = utils.flatten(resid.Subsets.dropna().to_list())
overlapping = list(set(overlapping))
overlapping

[]

In [12]:
# Deaths anywhere in the USA 
resid[resid.UUID.apply(lambda x: False if x in overlapping else True)][["Min", "Max",]].sum()
# resid[resid.UUID.apply(lambda x: False if x in overlapping else True)][["Min", "Max", "Areas"]].groupby(["Areas"]).sum()

Min    17
Max    52
dtype: object

#### Unused functions


In [ ]:
def check_identical(lst):
    groups = itertools.groupby(lst)
    return len(list(groups)) == 1

def get_gid_level_below(GID: str) -> tuple[str]:
    level = utils.get_level(GID)
    res = utils.gadm[utils.gadm[f"GID_{level}"] == GID][f"GID_{level+1}"]
    if res.dropna().unique().tolist:
        return tuple(sorted(res.dropna().unique().tolist()))

def intersection(query: tuple[str], space: tuple[str]):
    intersection = tuple(sorted(list(set(query) & set(space))))
    if intersection:
        return True
    return False

def get_gid_level_above(GID: str) -> str:
    res = GID.split(".")[:-1]
    if not res:
        return GID
    if len(res) == 1:
        return res[0]
    elif len(res) > 1:
        return f'{".".join(res)}_1'

def q_in_s(query: tuple[str], space: tuple[str]) -> tuple[str]:

    q_gid = sorted(utils.flatten([utils.elementary_gids(x) for x in list(query)]))
    s_gid = sorted(utils.flatten([utils.elementary_gids(x) for x in list(space)]))
    intr = sorted(list(set(q_gid) & set(s_gid)))
    if intr == q_gid:
        return True
    return False

In [ ]:
# query: all deaths in Montgomery County Pensyllvania at any time
groupbycols = ["ID", "Areas", "Breakdown", "Level"]
showcols = ["ID", "Min", "Max", "Areas", "Breakdown", "Level"]

deaths[deaths.apply(lambda row: utils.exact_match(utils.elementary_gids("USA.39_1"), space=row["Breakdown"]), axis=1)][showcols].groupby(groupbycols, as_index=False, dropna=False).sum()[showcols]

In [ ]:
deaths[deaths.apply(lambda row: utils.intersect(utils.elementary_gids("USA.39_1"), space=row["Breakdown"]), axis=1)][showcols].groupby(groupbycols, as_index=False, dropna=False).sum()[showcols]

In [ ]:
deaths[deaths.ID == 111].sort_values("Level", ascending=False)

In [ ]:
# start with the smallest levels
# for each GID:
    # check if GID is deepest level -- 'USA.10.15_1' is the deepest 
        # IF YES, STOP! No need to check overlap + ("subset_of" column is NaN and Approx remains False)

    # Get a list of levels lower than GID -- 'USA.10_1'
    # Check if any of these elements exist in any row of the same DATE and ID! 
    # if a match is found, compare to the original GID's min and max
        # IF min and max are smaller, write to column (subset_of = XID) and set Approx to True for the row being compared
        # IF min and max are larger, write to column (subset_of = XID, Approx = True) and write a new row with identical data except for the larger count. This row isn't subset_of anything and will be used in the final aggregation

In [ ]:
# try this:

# - check all records for regions or countries (aka Areas with GID_0)
    # - group by all other instances in the same ID and comapre the min and max 
    # - if the min and max are smaller, append a new record with a flag column set to True 
    # (the flag column "approx" will specify whether the number is directly from the article of inferred)
    # - add up this number to create an overall summary of the entire region (flag set to True), region = group of GID_0
    # - this results in an appended table where each Area (Breakdown) is summarized with GID_0 and a group of GID_0

# To query:
# - for any GID_1-GID_5, sum up the min and max for all records with an intersection with the Breakdown column 
# even if multiple finegrained records exist, we assume that each represents a separate incident 
# we could potentially be over-reporting the number so we should make it clear it's an estimate



In [ ]:
# try this:

# take the smallest GIDs in Areas (as small as reported), and check that they are smaller than every reported level above them